# IPL2023-Powerplay-Score-Prediction

## Importing libraries and data

In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [41]:
def log(*args):
    print('👉', *args)

In [42]:
ball_by_ball_orig = pd.read_csv('./Data/IPL_Ball_by_Ball_2008_2022.csv')
matches_result_orig = pd.read_csv('./Data/IPL_Matches_Result_2008_2022.csv')

In [43]:
ball_by_ball_orig.shape
ball_by_ball_orig.head()

(225954, 17)

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals
2,1312200,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,NaN,1,0,1,0,0,NaN,NaN,NaN,Rajasthan Royals
3,1312200,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
4,1312200,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals


In [44]:
matches_result_orig.shape
matches_result_orig.head()

(950, 20)

,ID,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,SuperOver,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2
0,1312200,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon
1,1312199,Ahmedabad,2022-05-27,2022,Qualifier 2,Royal Challengers Bangalore,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,field,N,Rajasthan Royals,Wickets,7.0,NaN,JC Buttler,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...",CB Gaffaney,Nitin Menon
2,1312198,Kolkata,2022-05-25,2022,Eliminator,Royal Challengers Bangalore,Lucknow Super Giants,"Eden Gardens, Kolkata",Lucknow Super Giants,field,N,Royal Challengers Bangalore,Runs,14.0,NaN,RM Patidar,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['Q de Kock', 'KL Rahul', 'M Vohra', 'DJ Hooda...",J Madanagopal,MA Gough
3,1312197,Kolkata,2022-05-24,2022,Qualifier 1,Rajasthan Royals,Gujarat Titans,"Eden Gardens, Kolkata",Gujarat Titans,field,N,Gujarat Titans,Wickets,7.0,NaN,DA Miller,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",BNJ Oxenford,VK Sharma
4,1304116,Mumbai,2022-05-22,2022,70,Sunrisers Hyderabad,Punjab Kings,"Wankhede Stadium, Mumbai",Sunrisers Hyderabad,bat,N,Punjab Kings,Wickets,5.0,NaN,Harpreet Brar,"['PK Garg', 'Abhishek Sharma', 'RA Tripathi', ...","['JM Bairstow', 'S Dhawan', 'M Shahrukh Khan',...",AK Chaudhary,NA Patwardhan


## Data cleaning

In [45]:
# Renaming and selecting columns for ball_by_ball DataFrame
ball_by_ball = ball_by_ball_orig.rename(columns={
    'ID': 'match_id',
    'ballnumber': 'ball_number',
    'non-striker': 'non_striker',
    'BattingTeam': 'batting_team',
}).loc[:, [
    'match_id',
    'innings',
    'batting_team',
    'overs',
    'ball_number',
    'batter',
    'bowler',
    'total_run',
]]

# Renaming and selecting columns for matches_result DataFrame
matches_result = matches_result_orig.rename(columns={
    'ID': 'match_id',
    'Team1': 'team_1',
    'Team2': 'team_2',
    'Venue': 'venue',
}).loc[:, [
    'match_id',
    'team_1',
    'team_2',
    'venue',
]]

In [46]:
ball_by_ball.head()

,match_id,innings,batting_team,overs,ball_number,batter,bowler,total_run
0,1312200,1,Rajasthan Royals,0,1,YBK Jaiswal,Mohammed Shami,0
1,1312200,1,Rajasthan Royals,0,2,YBK Jaiswal,Mohammed Shami,1
2,1312200,1,Rajasthan Royals,0,3,JC Buttler,Mohammed Shami,1
3,1312200,1,Rajasthan Royals,0,4,YBK Jaiswal,Mohammed Shami,0
4,1312200,1,Rajasthan Royals,0,5,YBK Jaiswal,Mohammed Shami,0


In [47]:
matches_result.head()

,match_id,team_1,team_2,venue
0,1312200,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad"
1,1312199,Royal Challengers Bangalore,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad"
2,1312198,Royal Challengers Bangalore,Lucknow Super Giants,"Eden Gardens, Kolkata"
3,1312197,Rajasthan Royals,Gujarat Titans,"Eden Gardens, Kolkata"
4,1304116,Sunrisers Hyderabad,Punjab Kings,"Wankhede Stadium, Mumbai"


## Team and venue names normalization

In [48]:
# https://www.iplt20.com/matches/schedule/men

venue_mapping_normal = {
    "Arun Jaitley Stadium": "Arun Jaitley Stadium",
    "Arun Jaitley Stadium, Delhi": "Arun Jaitley Stadium",
    "Feroz Shah Kotla": "Arun Jaitley Stadium",
    "Barsapara Cricket Stadium": "Barsapara Cricket Stadium",
    "Barsapara Cricket Stadium, Guwahati": "Barsapara Cricket Stadium",
    "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium": "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium",
    "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium, Lucknow": "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium",
    "Eden Gardens": "Eden Gardens",
    "Eden Gardens, Kolkata": "Eden Gardens",
    "Himachal Pradesh Cricket Association Stadium": "Himachal Pradesh Cricket Association Stadium",
    "Himachal Pradesh Cricket Association Stadium, Dharamsala": "Himachal Pradesh Cricket Association Stadium",
    "M Chinnaswamy Stadium": "M Chinnaswamy Stadium",
    "M Chinnaswamy Stadium, Bengaluru": "M Chinnaswamy Stadium",
    "M Chinnaswamy Stadium, Bangalore": "M Chinnaswamy Stadium",
    "M.Chinnaswamy Stadium": "M Chinnaswamy Stadium",
    "M.Chinnaswamy Stadium, Bengaluru": "M Chinnaswamy Stadium",
    "M.Chinnaswamy Stadium, Bangalore": "M Chinnaswamy Stadium",
    "MA Chidambaram Stadium": "MA Chidambaram Stadium",
    "MA Chidambaram Stadium, Chennai": "MA Chidambaram Stadium",
    "MA Chidambaram Stadium, Chepauk": "MA Chidambaram Stadium",
    "MA Chidambaram Stadium, Chepauk, Chennai": "MA Chidambaram Stadium",
    "Narendra Modi Stadium": "Narendra Modi Stadium",
    "Narendra Modi Stadium, Ahmedabad": "Narendra Modi Stadium",
    "Punjab Cricket Association IS Bindra Stadium": "Punjab Cricket Association IS Bindra Stadium",
    "Punjab Cricket Association IS Bindra Stadium, Mohali": "Punjab Cricket Association IS Bindra Stadium",
    "Punjab Cricket Association Stadium, Mohali": "Punjab Cricket Association IS Bindra Stadium",
    "Rajiv Gandhi International Stadium": "Rajiv Gandhi International Stadium",
    "Rajiv Gandhi International Stadium, Hyderabad": "Rajiv Gandhi International Stadium",
    "Rajiv Gandhi International Stadium, Uppal": "Rajiv Gandhi International Stadium",
    "Sawai Mansingh Stadium": "Sawai Mansingh Stadium",
    "Sawai Mansingh Stadium, Jaipur": "Sawai Mansingh Stadium",
    "Wankhede Stadium": "Wankhede Stadium",
    "Wankhede Stadium, Mumbai": "Wankhede Stadium"
}

In [49]:
venue_mapping_kebab = {
    "arun-jaitley-stadium": "Arun Jaitley Stadium",
    "arun-jaitley-stadium-delhi": "Arun Jaitley Stadium",
    "feroz-shah-kotla": "Arun Jaitley Stadium",
    "barsapara-cricket-stadium": "Barsapara Cricket Stadium",
    "barsapara-cricket-stadium-guwahati": "Barsapara Cricket Stadium",
    "bharat-ratna-shri-atal-bihari-vajpayee-ekana-cricket-stadium": "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium",
    "bharat-ratna-shri-atal-bihari-vajpayee-ekana-cricket-stadium-lucknow": "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium",
    "eden-gardens": "Eden Gardens",
    "eden-gardens-kolkata": "Eden Gardens",
    "himachal-pradesh-cricket-association-stadium": "Himachal Pradesh Cricket Association Stadium",
    "himachal-pradesh-cricket-association-stadium-dharamsala": "Himachal Pradesh Cricket Association Stadium",
    "m-chinnaswamy-stadium": "M Chinnaswamy Stadium",
    "m-chinnaswamy-stadium-bengaluru": "M Chinnaswamy Stadium",
    "m-chinnaswamy-stadium-bangalore": "M Chinnaswamy Stadium",
    "mchinnaswamy-stadium": "M Chinnaswamy Stadium",
    "mchinnaswamy-stadium-bengaluru": "M Chinnaswamy Stadium",
    "mchinnaswamy-stadium-bangalore": "M Chinnaswamy Stadium",
    "ma-chidambaram-stadium": "MA Chidambaram Stadium",
    "ma-chidambaram-stadium-chennai": "MA Chidambaram Stadium",
    "ma-chidambaram-stadium-chepauk": "MA Chidambaram Stadium",
    "ma-chidambaram-stadium-chepauk-chennai": "MA Chidambaram Stadium",
    "narendra-modi-stadium": "Narendra Modi Stadium",
    "narendra-modi-stadium-ahmedabad": "Narendra Modi Stadium",
    "punjab-cricket-association-is-bindra-stadium": "Punjab Cricket Association IS Bindra Stadium",
    "punjab-cricket-association-is-bindra-stadium-mohali": "Punjab Cricket Association IS Bindra Stadium",
    "punjab-cricket-association-stadium-mohali": "Punjab Cricket Association IS Bindra Stadium",
    "rajiv-gandhi-international-stadium": "Rajiv Gandhi International Stadium",
    "rajiv-gandhi-international-stadium-hyderabad": "Rajiv Gandhi International Stadium",
    "rajiv-gandhi-international-stadium-uppal": "Rajiv Gandhi International Stadium",
    "sawai-mansingh-stadium": "Sawai Mansingh Stadium",
    "sawai-mansingh-stadium-jaipur": "Sawai Mansingh Stadium",
    "wankhede-stadium": "Wankhede Stadium",
    "wankhede-stadium-mumbai": "Wankhede Stadium"
}

In [50]:
venue_mapping_tags = {
    "delhi": "Arun Jaitley Stadium",
    "arun jaitley": "Arun Jaitley Stadium",
    "guwahati": "Barsapara Cricket Stadium",
    "barsapara": "Barsapara Cricket Stadium",
    "bhupen hazarika": "Barsapara Cricket Stadium",
    "assam cricket association stadium": "Barsapara Cricket Stadium",
    "lucknow": "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium",
    "ekana": "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium",
    "atal bihari": "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium",
    "kolkata": "Eden Gardens",
    "eden gardens": "Eden Gardens",
    "dharamsala": "Himachal Pradesh Cricket Association Stadium",
    "himachal pradesh": "Himachal Pradesh Cricket Association Stadium",
    "bengaluru": "M Chinnaswamy Stadium",
    "bengalore": "M Chinnaswamy Stadium",
    "chinnaswamy": "M Chinnaswamy Stadium",
    "chennai": "MA Chidambaram Stadium",
    "chepauk": "MA Chidambaram Stadium",
    "chidambaram": "MA Chidambaram Stadium",
    "ahmedabad": "Narendra Modi Stadium",
    "narendra modi": "Narendra Modi Stadium",
    "mohali": "Punjab Cricket Association IS Bindra Stadium",
    "punjab cricket association": "Punjab Cricket Association IS Bindra Stadium",
    "is bindra": "Punjab Cricket Association IS Bindra Stadium",
    "hyderabad": "Rajiv Gandhi International Stadium",
    "rajiv gandhi": "Rajiv Gandhi International Stadium",
    "jaipur": "Sawai Mansingh Stadium",
    "sawai mansingh": "Sawai Mansingh Stadium",
    "mumbai": "Wankhede Stadium",
    "wankhede": "Wankhede Stadium"
}

In [51]:
# venues in `matches_result` which aren't present in `venue_mapping_normal.keys`,
# collectively considered as `Other` venue after mapping
np.setdiff1d(matches_result.venue.unique(), list(venue_mapping_normal.keys()))

array(['Barabati Stadium', 'Brabourne Stadium',
       'Brabourne Stadium, Mumbai', 'Buffalo Park',
       'De Beers Diamond Oval', 'Dr DY Patil Sports Academy',
       'Dr DY Patil Sports Academy, Mumbai',
       'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
       'Dubai International Cricket Stadium', 'Green Park',
       'Holkar Cricket Stadium', 'JSCA International Stadium Complex',
       'Kingsmead', 'Maharashtra Cricket Association Stadium',
       'Maharashtra Cricket Association Stadium, Pune', 'Nehru Stadium',
       'New Wanderers Stadium', 'Newlands', 'OUTsurance Oval',
       'Sardar Patel Stadium, Motera',
       'Saurashtra Cricket Association Stadium',
       'Shaheed Veer Narayan Singh International Stadium',
       'Sharjah Cricket Stadium', 'Sheikh Zayed Stadium',
       "St George's Park", 'Subrata Roy Sahara Stadium',
       'SuperSport Park', 'Vidarbha Cricket Association Stadium, Jamtha',
       'Zayed Cricket Stadium, Abu Dhabi'], dtype=object)

In [52]:
set(matches_result['team_1'].unique()) == set(
    matches_result['team_2'].unique()) == set(ball_by_ball['batting_team'].unique())

True

In [53]:
team_mapping = {
    'Rajasthan Royals': 'Rajasthan Royals',
    'Gujarat Titans': 'Gujarat Titans',
    'Royal Challengers Bangalore': 'Royal Challengers Bangalore',
    'Lucknow Super Giants': 'Lucknow Super Giants',
    'Sunrisers Hyderabad': 'Sunrisers Hyderabad',
    'Mumbai Indians': 'Mumbai Indians',
    'Chennai Super Kings': 'Chennai Super Kings',
    'Kolkata Knight Riders': 'Kolkata Knight Riders',
    'Kings XI Punjab': 'Punjab Kings',
    'Punjab Kings': 'Punjab Kings',
    'Delhi Daredevils': 'Delhi Capitals',
    'Delhi Capitals': 'Delhi Capitals',
}

In [54]:
# teams in `team_mapping.keys` which aren't present in `matches_result`
# before mapping: [], after mapping: ['Delhi Daredevils', 'Kings XI Punjab']
np.setdiff1d(
    list(team_mapping.keys()), matches_result['team_1'].unique()
)

array([], dtype='<U27')

In [55]:
# teams in `matches_result` which aren't present in `team_mapping.keys`,
# collectively considered as `Other` team after mapping
np.setdiff1d(
    matches_result['team_1'].unique(), list(team_mapping.keys())
)

array(['Deccan Chargers', 'Gujarat Lions', 'Kochi Tuskers Kerala',
       'Pune Warriors', 'Rising Pune Supergiant',
       'Rising Pune Supergiants'], dtype=object)

## Apply team and venue mapping

In [56]:
matches_result.venue = matches_result.venue.map(
    venue_mapping_normal).fillna('Other')

In [57]:
matches_result.team_1 = matches_result.team_1.map(team_mapping).fillna('Other')
matches_result.team_2 = matches_result.team_2.map(team_mapping).fillna('Other')

In [58]:
ball_by_ball.batting_team = ball_by_ball.batting_team.map(
    team_mapping).fillna('Other')

## Mapping verification

In [59]:
# new venues in ipl23: ['Barsapara Cricket Stadium', 'Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium']

list_of_ipl23_venues = ['Arun Jaitley Stadium', 'Barsapara Cricket Stadium',
                        'Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium',
                        'Eden Gardens', 'Himachal Pradesh Cricket Association Stadium',
                        'M Chinnaswamy Stadium', 'MA Chidambaram Stadium',
                        'Narendra Modi Stadium',
                        'Punjab Cricket Association IS Bindra Stadium',
                        'Rajiv Gandhi International Stadium', 'Sawai Mansingh Stadium',
                        'Wankhede Stadium']

list_of_ipl23_teams = ['Chennai Super Kings', 'Delhi Capitals', 'Gujarat Titans',
                       'Kolkata Knight Riders', 'Lucknow Super Giants', 'Mumbai Indians',
                       'Punjab Kings', 'Rajasthan Royals', 'Royal Challengers Bangalore',
                       'Sunrisers Hyderabad']

In [60]:
# list_of_ipl23_venues == venue_mapping_normal
np.array_equal(np.unique(
    list(venue_mapping_normal.values())
), list_of_ipl23_venues)

# list_of_ipl23_teams == team_mapping
np.array_equal(np.unique(
    list(team_mapping.values())
), list_of_ipl23_teams)

True

True

## Normalize overs and innings in `ball_by_ball`

In [61]:
ball_by_ball['overs'].unique()
ball_by_ball['innings'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19], dtype=int64)

array([1, 2, 3, 4, 5, 6], dtype=int64)

In [62]:
ball_by_ball = ball_by_ball.loc[(
    ball_by_ball.overs <= 5) & (ball_by_ball.innings <= 2)]

## ⭐ Integration of `ball_by_ball` and `matches_result`

In [63]:
ball_by_ball

,match_id,innings,batting_team,overs,ball_number,batter,bowler,total_run
0,1312200,1,Rajasthan Royals,0,1,YBK Jaiswal,Mohammed Shami,0
1,1312200,1,Rajasthan Royals,0,2,YBK Jaiswal,Mohammed Shami,1
2,1312200,1,Rajasthan Royals,0,3,JC Buttler,Mohammed Shami,1
3,1312200,1,Rajasthan Royals,0,4,YBK Jaiswal,Mohammed Shami,0
4,1312200,1,Rajasthan Royals,0,5,YBK Jaiswal,Mohammed Shami,0
...,...,...,...,...,...,...,...,...
225886,335982,2,Royal Challengers Bangalore,5,2,W Jaffer,AB Dinda,0
225887,335982,2,Royal Challengers Bangalore,5,3,MV Boucher,AB Dinda,0
225888,335982,2,Royal Challengers Bangalore,5,4,MV Boucher,AB Dinda,1
225889,335982,2,Royal Challengers Bangalore,5,5,CL White,AB Dinda,1


In [64]:
matches_result

,match_id,team_1,team_2,venue
0,1312200,Rajasthan Royals,Gujarat Titans,Narendra Modi Stadium
1,1312199,Royal Challengers Bangalore,Rajasthan Royals,Narendra Modi Stadium
2,1312198,Royal Challengers Bangalore,Lucknow Super Giants,Eden Gardens
3,1312197,Rajasthan Royals,Gujarat Titans,Eden Gardens
4,1304116,Sunrisers Hyderabad,Punjab Kings,Wankhede Stadium
...,...,...,...,...
945,335986,Kolkata Knight Riders,Other,Eden Gardens
946,335985,Mumbai Indians,Royal Challengers Bangalore,Wankhede Stadium
947,335984,Delhi Capitals,Rajasthan Royals,Arun Jaitley Stadium
948,335983,Punjab Kings,Chennai Super Kings,Punjab Cricket Association IS Bindra Stadium


In [65]:
len(ball_by_ball['match_id'].unique())

# To find the matches: second inning data is missing
for match_id in ball_by_ball['match_id'].unique():
    l1 = len(ball_by_ball.query(f'match_id == {
             match_id}').query('innings == 1'))
    l2 = len(ball_by_ball.query(f'match_id == {
             match_id}').query('innings == 2'))

    if l1 == 0 or l2 == 0:
        print(match_id, l1, l2)
        continue

950

829763 38 0
501265 37 0


In [66]:
ball_by_ball.loc[(ball_by_ball['match_id'] == 1312200)
                 & (ball_by_ball['innings'] == 1)]

ball_by_ball.loc[(ball_by_ball['match_id'] == 1312200)
                 & (ball_by_ball['innings'] == 1)].agg({
                     'total_run': 'sum',
                     'batter': 'unique',
                     'bowler': 'unique',
                 })

,match_id,innings,batting_team,overs,ball_number,batter,bowler,total_run
0,1312200,1,Rajasthan Royals,0,1,YBK Jaiswal,Mohammed Shami,0
1,1312200,1,Rajasthan Royals,0,2,YBK Jaiswal,Mohammed Shami,1
2,1312200,1,Rajasthan Royals,0,3,JC Buttler,Mohammed Shami,1
3,1312200,1,Rajasthan Royals,0,4,YBK Jaiswal,Mohammed Shami,0
4,1312200,1,Rajasthan Royals,0,5,YBK Jaiswal,Mohammed Shami,0
5,1312200,1,Rajasthan Royals,0,6,YBK Jaiswal,Mohammed Shami,0
6,1312200,1,Rajasthan Royals,1,1,JC Buttler,Yash Dayal,0
7,1312200,1,Rajasthan Royals,1,2,JC Buttler,Yash Dayal,0
8,1312200,1,Rajasthan Royals,1,3,JC Buttler,Yash Dayal,4
9,1312200,1,Rajasthan Royals,1,4,JC Buttler,Yash Dayal,0


total_run                                                   44
batter                    [YBK Jaiswal, JC Buttler, SV Samson]
bowler       [Mohammed Shami, Yash Dayal, LH Ferguson, Rash...
dtype: object

In [67]:
grouped_data = ball_by_ball.groupby(['match_id', 'innings', 'batting_team'])
grouped_data = [(name, group) for name, group in grouped_data]

len(grouped_data)

name, group = [(name, group) for name, group in grouped_data][0]
name
group

1898

(335982, 1, 'Kolkata Knight Riders')

,match_id,innings,batting_team,overs,ball_number,batter,bowler,total_run
225729,335982,1,Kolkata Knight Riders,0,1,SC Ganguly,P Kumar,1
225730,335982,1,Kolkata Knight Riders,0,2,BB McCullum,P Kumar,0
225731,335982,1,Kolkata Knight Riders,0,3,BB McCullum,P Kumar,1
225732,335982,1,Kolkata Knight Riders,0,4,BB McCullum,P Kumar,0
225733,335982,1,Kolkata Knight Riders,0,5,BB McCullum,P Kumar,0
225734,335982,1,Kolkata Knight Riders,0,6,BB McCullum,P Kumar,0
225735,335982,1,Kolkata Knight Riders,0,7,BB McCullum,P Kumar,1
225736,335982,1,Kolkata Knight Riders,1,1,BB McCullum,Z Khan,0
225737,335982,1,Kolkata Knight Riders,1,2,BB McCullum,Z Khan,4
225738,335982,1,Kolkata Knight Riders,1,3,BB McCullum,Z Khan,4


## Making data

In [121]:
grouped_data = ball_by_ball.groupby(['match_id', 'innings', 'batting_team'])

'After Aggregation'
grouped_data.agg({
    'total_run': 'sum',
    'batter': 'unique',
    'bowler': 'unique',
})

'After aggregation plus reset index'
grouped_data.agg({
    'total_run': 'sum',
    'batter': 'unique',
    'bowler': 'unique',
}).reset_index().rename(columns={
    'batter': 'batsmen', 'bowler': 'bowlers', 'total_run': 'total_runs'
})

data = grouped_data.agg({
    'total_run': 'sum',
    'batter': 'unique',
    'bowler': 'unique',
}).reset_index().rename(columns={
    'batter': 'batsmen', 'bowler': 'bowlers', 'total_run': 'total_runs'
}).merge(matches_result, on=['match_id'])

'Data: After merging with matches result'
data

'After Aggregation'

total_run  \
match_id innings batting_team                             
335982   1       Kolkata Knight Riders               61   
         2       Royal Challengers Bangalore         26   
335983   1       Chennai Super Kings                 53   
         2       Punjab Kings                        63   
335984   1       Rajasthan Royals                    40   
...                                                 ...   
1312198  2       Lucknow Super Giants                62   
1312199  1       Royal Challengers Bangalore         46   
         2       Rajasthan Royals                    67   
1312200  1       Rajasthan Royals                    44   
         2       Gujarat Titans                      31   

                                                                                         batter  \
match_id innings batting_team                                                                     
335982   1       Kolkata Knight Riders                    [SC Ganguly, BB McCullum, RT Ponting]   
         2       Royal Challengers Bangalore  [R Dravid, W Jaffer, V Kohli, JH Kallis, CL Wh...   
335983   1       Chennai Super Kings                          [PA Patel, ML Hayden, MEK Hussey]   
         2       Punjab Kings                                                [K Goel, JR Hopes]   
335984   1       Rajasthan Royals                       [T Kohli, YK Pathan, SR Watson, M Kaif]   
...                                                                                         ...   
1312198  2       Lucknow Super Giants                  [Q de Kock, KL Rahul, M Vohra, DJ Hooda]   
1312199  1       Royal Challengers Bangalore                [V Kohli, F du Plessis, RM Patidar]   
         2       Rajasthan Royals                          [YBK Jaiswal, JC Buttler, SV Samson]   
1312200  1       Rajasthan Royals                          [YBK Jaiswal, JC Buttler, SV Samson]   
         2       Gujarat Titans                     [WP Saha, Shubman Gill, MS Wade, HH Pandya]   

                                                                                         bowler  
match_id innings batting_team                                                                    
335982   1       Kolkata Knight Riders                             [P Kumar, Z Khan, AA Noffke]  
         2       Royal Challengers Bangalore                   [AB Dinda, I Sharma, AB Agarkar]  
335983   1       Chennai Super Kings                             [B Lee, S Sreesanth, JR Hopes]  
         2       Punjab Kings                                               [JDP Oram, MS Gony]  
335984   1       Rajasthan Royals                           [GD McGrath, B Geeves, MF Maharoof]  
...                                                                                         ...  
1312198  2       Lucknow Super Giants             [Mohammed Siraj, JR Hazlewood, Shahbaz Ahmed]  
1312199  1       Royal Challengers Bangalore                      [TA Boult, M Prasidh Krishna]  
         2       Rajasthan Royals             [Mohammed Siraj, JR Hazlewood, GJ Maxwell, Sha...  
1312200  1       Rajasthan Royals             [Mohammed Shami, Yash Dayal, LH Ferguson, Rash...  
         2       Gujarat Titans                        [TA Boult, M Prasidh Krishna, YS Chahal]  

[1898 rows x 3 columns]

'After aggregation plus reset index'

,match_id,innings,batting_team,total_runs,batsmen,bowlers
0,335982,1,Kolkata Knight Riders,61,"[SC Ganguly, BB McCullum, RT Ponting]","[P Kumar, Z Khan, AA Noffke]"
1,335982,2,Royal Challengers Bangalore,26,"[R Dravid, W Jaffer, V Kohli, JH Kallis, CL Wh...","[AB Dinda, I Sharma, AB Agarkar]"
2,335983,1,Chennai Super Kings,53,"[PA Patel, ML Hayden, MEK Hussey]","[B Lee, S Sreesanth, JR Hopes]"
3,335983,2,Punjab Kings,63,"[K Goel, JR Hopes]","[JDP Oram, MS Gony]"
4,335984,1,Rajasthan Royals,40,"[T Kohli, YK Pathan, SR Watson, M Kaif]","[GD McGrath, B Geeves, MF Maharoof]"
...,...,...,...,...,...,...
1893,1312198,2,Lucknow Super Giants,62,"[Q de Kock, KL Rahul, M Vohra, DJ Hooda]","[Mohammed Siraj, JR Hazlewood, Shahbaz Ahmed]"
1894,1312199,1,Royal Challengers Bangalore,46,"[V Kohli, F du Plessis, RM Patidar]","[TA Boult, M Prasidh Krishna]"
1895,1312199,2,Rajasthan Royals,67,"[YBK Jaiswal, JC Buttler, SV Samson]","[Mohammed Siraj, JR Hazlewood, GJ Maxwell, Sha..."
1896,1312200,1,Rajasthan Royals,44,"[YBK Jaiswal, JC Buttler, SV Samson]","[Mohammed Shami, Yash Dayal, LH Ferguson, Rash..."


'Data: After merging with matches result'

,match_id,innings,batting_team,total_runs,batsmen,bowlers,team_1,team_2,venue
0,335982,1,Kolkata Knight Riders,61,"[SC Ganguly, BB McCullum, RT Ponting]","[P Kumar, Z Khan, AA Noffke]",Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium
1,335982,2,Royal Challengers Bangalore,26,"[R Dravid, W Jaffer, V Kohli, JH Kallis, CL Wh...","[AB Dinda, I Sharma, AB Agarkar]",Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium
2,335983,1,Chennai Super Kings,53,"[PA Patel, ML Hayden, MEK Hussey]","[B Lee, S Sreesanth, JR Hopes]",Punjab Kings,Chennai Super Kings,Punjab Cricket Association IS Bindra Stadium
3,335983,2,Punjab Kings,63,"[K Goel, JR Hopes]","[JDP Oram, MS Gony]",Punjab Kings,Chennai Super Kings,Punjab Cricket Association IS Bindra Stadium
4,335984,1,Rajasthan Royals,40,"[T Kohli, YK Pathan, SR Watson, M Kaif]","[GD McGrath, B Geeves, MF Maharoof]",Delhi Capitals,Rajasthan Royals,Arun Jaitley Stadium
...,...,...,...,...,...,...,...,...,...
1893,1312198,2,Lucknow Super Giants,62,"[Q de Kock, KL Rahul, M Vohra, DJ Hooda]","[Mohammed Siraj, JR Hazlewood, Shahbaz Ahmed]",Royal Challengers Bangalore,Lucknow Super Giants,Eden Gardens
1894,1312199,1,Royal Challengers Bangalore,46,"[V Kohli, F du Plessis, RM Patidar]","[TA Boult, M Prasidh Krishna]",Royal Challengers Bangalore,Rajasthan Royals,Narendra Modi Stadium
1895,1312199,2,Rajasthan Royals,67,"[YBK Jaiswal, JC Buttler, SV Samson]","[Mohammed Siraj, JR Hazlewood, GJ Maxwell, Sha...",Royal Challengers Bangalore,Rajasthan Royals,Narendra Modi Stadium
1896,1312200,1,Rajasthan Royals,44,"[YBK Jaiswal, JC Buttler, SV Samson]","[Mohammed Shami, Yash Dayal, LH Ferguson, Rash...",Rajasthan Royals,Gujarat Titans,Narendra Modi Stadium


## Add a new column `bowling_team`

In [122]:
mask = data['batting_team'] == data['team_1']
len(data[mask])

mask = data['batting_team'] == data['team_2']
len(data[mask])

960 + 958 - 1898

960

958

20

In [123]:
s = data[(data['batting_team'] == 'Other') & (data['team_1']
                                              == 'Other') & (data['team_2'] == 'Other')]
len(s)
s

20

,match_id,innings,batting_team,total_runs,batsmen,bowlers,team_1,team_2,venue
368,501207,1,Other,37,"[BB McCullum, PA Patel, VVS Laxman, DPMD Jayaw...","[AC Thomas, SB Wagh, WD Parnell]",Other,Other,Other
369,501207,2,Other,44,"[JD Ryder, GC Smith, M Manhas]","[RP Singh, S Sreesanth, M Muralitharan, R Vina...",Other,Other,Other
410,501229,1,Other,20,"[S Sohal, S Dhawan, B Chipli, KC Sangakkara]","[RP Singh, S Sreesanth, R Vinay Kumar]",Other,Other,Other
411,501229,2,Other,29,"[BB McCullum, PA Patel, DPMD Jayawardene, RV G...","[DW Steyn, I Sharma]",Other,Other,Other
452,501250,1,Other,50,"[S Dhawan, DB Ravi Teja]","[B Kumar, SB Wagh, WD Parnell, R Sharma]",Other,Other,Rajiv Gandhi International Stadium
453,501250,2,Other,52,"[JD Ryder, MK Pandey]","[DW Steyn, I Sharma, PP Ojha, JP Duminy, DT Ch...",Other,Other,Rajiv Gandhi International Stadium
470,501259,1,Other,39,"[JD Ryder, MK Pandey, SC Ganguly, RV Uthappa]","[JP Duminy, DW Steyn, I Sharma, DT Christian]",Other,Other,Other
471,501259,2,Other,57,"[S Sohal, S Dhawan]","[B Kumar, WD Parnell, R Sharma, MR Marsh]",Other,Other,Other
561,548341,1,Other,50,"[PA Patel, S Dhawan, CL White]","[A Nehra, AC Thomas, MN Samuels, B Kumar]",Other,Other,Other
562,548341,2,Other,44,"[MK Pandey, JD Ryder, SC Ganguly, RV Uthappa]","[Ankit Sharma, JP Duminy, MS Gony, J Theron]",Other,Other,Other


In [124]:
mask = data['batting_team'] == data['team_1']
data.loc[mask, 'bowling_team'] = data['team_2']
data.loc[~mask, 'bowling_team'] = data['team_1']

In [125]:
data

,match_id,innings,batting_team,total_runs,batsmen,bowlers,team_1,team_2,venue,bowling_team
0,335982,1,Kolkata Knight Riders,61,"[SC Ganguly, BB McCullum, RT Ponting]","[P Kumar, Z Khan, AA Noffke]",Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,Royal Challengers Bangalore
1,335982,2,Royal Challengers Bangalore,26,"[R Dravid, W Jaffer, V Kohli, JH Kallis, CL Wh...","[AB Dinda, I Sharma, AB Agarkar]",Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,Kolkata Knight Riders
2,335983,1,Chennai Super Kings,53,"[PA Patel, ML Hayden, MEK Hussey]","[B Lee, S Sreesanth, JR Hopes]",Punjab Kings,Chennai Super Kings,Punjab Cricket Association IS Bindra Stadium,Punjab Kings
3,335983,2,Punjab Kings,63,"[K Goel, JR Hopes]","[JDP Oram, MS Gony]",Punjab Kings,Chennai Super Kings,Punjab Cricket Association IS Bindra Stadium,Chennai Super Kings
4,335984,1,Rajasthan Royals,40,"[T Kohli, YK Pathan, SR Watson, M Kaif]","[GD McGrath, B Geeves, MF Maharoof]",Delhi Capitals,Rajasthan Royals,Arun Jaitley Stadium,Delhi Capitals
...,...,...,...,...,...,...,...,...,...,...
1893,1312198,2,Lucknow Super Giants,62,"[Q de Kock, KL Rahul, M Vohra, DJ Hooda]","[Mohammed Siraj, JR Hazlewood, Shahbaz Ahmed]",Royal Challengers Bangalore,Lucknow Super Giants,Eden Gardens,Royal Challengers Bangalore
1894,1312199,1,Royal Challengers Bangalore,46,"[V Kohli, F du Plessis, RM Patidar]","[TA Boult, M Prasidh Krishna]",Royal Challengers Bangalore,Rajasthan Royals,Narendra Modi Stadium,Rajasthan Royals
1895,1312199,2,Rajasthan Royals,67,"[YBK Jaiswal, JC Buttler, SV Samson]","[Mohammed Siraj, JR Hazlewood, GJ Maxwell, Sha...",Royal Challengers Bangalore,Rajasthan Royals,Narendra Modi Stadium,Royal Challengers Bangalore
1896,1312200,1,Rajasthan Royals,44,"[YBK Jaiswal, JC Buttler, SV Samson]","[Mohammed Shami, Yash Dayal, LH Ferguson, Rash...",Rajasthan Royals,Gujarat Titans,Narendra Modi Stadium,Gujarat Titans


## Remove useless samples from data

In [126]:
'data for one innings is missing'
data.query('match_id == 829763')

'data for one innings is missing'
data.query('match_id == 501265')

'total_runs for one innings is 2 (probably a mistake in data entry)'
data.query('match_id == 829813')

'data for one innings is missing'

,match_id,innings,batting_team,total_runs,batsmen,bowlers,team_1,team_2,venue,bowling_team
971,829763,1,Royal Challengers Bangalore,52,"[CH Gayle, AB de Villiers, V Kohli, Mandeep Si...","[TG Southee, DS Kulkarni, JP Faulkner, SR Watson]",Royal Challengers Bangalore,Rajasthan Royals,M Chinnaswamy Stadium,Rajasthan Royals


'data for one innings is missing'

,match_id,innings,batting_team,total_runs,batsmen,bowlers,team_1,team_2,venue,bowling_team
482,501265,1,Delhi Capitals,33,"[NV Ojha, DA Warner, MS Wade]","[AC Thomas, JP Faulkner, SS Mundhe]",Delhi Capitals,Other,Arun Jaitley Stadium,Other


'total_runs for one innings is 2 (probably a mistake in data entry)'

,match_id,innings,batting_team,total_runs,batsmen,bowlers,team_1,team_2,venue,bowling_team
1020,829813,1,Delhi Capitals,54,"[Q de Kock, SS Iyer]","[MA Starc, AB Dinda, HV Patel, D Wiese]",Royal Challengers Bangalore,Delhi Capitals,M Chinnaswamy Stadium,Royal Challengers Bangalore
1021,829813,2,Royal Challengers Bangalore,2,"[V Kohli, CH Gayle]","[J Yadav, Z Khan]",Royal Challengers Bangalore,Delhi Capitals,M Chinnaswamy Stadium,Delhi Capitals


In [127]:
data = data.drop(data[
    (data['match_id'] == 829763) | (data['match_id']
                                    == 501265) | (data['match_id'] == 829813)
].index)

## Get count of batsmen & bowlers for each innings

In [128]:
data['count_batsmen'] = [len(x) for x in data['batsmen']]
data['count_bowlers'] = [len(x) for x in data['bowlers']]

## Finalizing data

In [129]:
data = data[
    ['venue', 'innings', 'batting_team', 'bowling_team',
        'count_batsmen', 'count_bowlers', 'total_runs']
].reset_index(drop=True)

In [130]:
data

,venue,innings,batting_team,bowling_team,count_batsmen,count_bowlers,total_runs
0,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,3,3,61
1,M Chinnaswamy Stadium,2,Royal Challengers Bangalore,Kolkata Knight Riders,6,3,26
2,Punjab Cricket Association IS Bindra Stadium,1,Chennai Super Kings,Punjab Kings,3,3,53
3,Punjab Cricket Association IS Bindra Stadium,2,Punjab Kings,Chennai Super Kings,2,2,63
4,Arun Jaitley Stadium,1,Rajasthan Royals,Delhi Capitals,4,3,40
...,...,...,...,...,...,...,...
1889,Eden Gardens,2,Lucknow Super Giants,Royal Challengers Bangalore,4,3,62
1890,Narendra Modi Stadium,1,Royal Challengers Bangalore,Rajasthan Royals,3,2,46
1891,Narendra Modi Stadium,2,Rajasthan Royals,Royal Challengers Bangalore,3,4,67
1892,Narendra Modi Stadium,1,Rajasthan Royals,Gujarat Titans,3,4,44


## Train test splitting

In [173]:
data['venue'].value_counts()
data['batting_team'].value_counts()
[len(group) for name, group in data.groupby(['venue', 'batting_team'])].count(1)

venue
Other                                           718
Wankhede Stadium                                208
Eden Gardens                                    158
M Chinnaswamy Stadium                           156
Arun Jaitley Stadium                            154
MA Chidambaram Stadium                          134
Rajiv Gandhi International Stadium              128
Punjab Cricket Association IS Bindra Stadium    112
Sawai Mansingh Stadium                           94
Himachal Pradesh Cricket Association Stadium     18
Narendra Modi Stadium                            14
Name: count, dtype: int64

batting_team
Mumbai Indians                 231
Royal Challengers Bangalore    224
Kolkata Knight Riders          223
Delhi Capitals                 222
Punjab Kings                   218
Chennai Super Kings            208
Other                          194
Rajasthan Royals               191
Sunrisers Hyderabad            152
Gujarat Titans                  16
Lucknow Super Giants            15
Name: count, dtype: int64

5

In [174]:
from sklearn.model_selection import train_test_split

# List of columns for stratified sampling
stratify_cols = ['batting_team']

# Perform stratified sampling
train_data, val_data = train_test_split(
    data, test_size=0.2, stratify=data[stratify_cols], random_state=42)

In [175]:
train_data['venue'].value_counts()
val_data['venue'].value_counts()

train_data['batting_team'].value_counts()
val_data['batting_team'].value_counts()

venue
Other                                           585
Wankhede Stadium                                164
Eden Gardens                                    128
Arun Jaitley Stadium                            119
M Chinnaswamy Stadium                           117
MA Chidambaram Stadium                          108
Rajiv Gandhi International Stadium              105
Punjab Cricket Association IS Bindra Stadium     86
Sawai Mansingh Stadium                           79
Himachal Pradesh Cricket Association Stadium     14
Narendra Modi Stadium                            10
Name: count, dtype: int64

venue
Other                                           133
Wankhede Stadium                                 44
M Chinnaswamy Stadium                            39
Arun Jaitley Stadium                             35
Eden Gardens                                     30
Punjab Cricket Association IS Bindra Stadium     26
MA Chidambaram Stadium                           26
Rajiv Gandhi International Stadium               23
Sawai Mansingh Stadium                           15
Narendra Modi Stadium                             4
Himachal Pradesh Cricket Association Stadium      4
Name: count, dtype: int64

batting_team
Mumbai Indians                 185
Royal Challengers Bangalore    179
Kolkata Knight Riders          178
Delhi Capitals                 178
Punjab Kings                   174
Chennai Super Kings            166
Other                          155
Rajasthan Royals               153
Sunrisers Hyderabad            122
Gujarat Titans                  13
Lucknow Super Giants            12
Name: count, dtype: int64

batting_team
Mumbai Indians                 46
Royal Challengers Bangalore    45
Kolkata Knight Riders          45
Punjab Kings                   44
Delhi Capitals                 44
Chennai Super Kings            42
Other                          39
Rajasthan Royals               38
Sunrisers Hyderabad            30
Gujarat Titans                  3
Lucknow Super Giants            3
Name: count, dtype: int64

## Data preprocessing before training model

In [176]:
train_X = train_data.drop(columns=['total_runs'])
train_y = train_data["total_runs"]

val_X = val_data.drop(columns=['total_runs'])
val_y = val_data["total_runs"]

## ⭐ Encoding of categorical inputs and feature scaling

In [132]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [133]:
venue_categories = list_of_ipl23_venues + ['Other']
team_categories = list_of_ipl23_teams + ['Other']

In [134]:
len(venue_categories)
venue_categories

len(team_categories)
team_categories

13

['Arun Jaitley Stadium',
 'Barsapara Cricket Stadium',
 'Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium',
 'Eden Gardens',
 'Himachal Pradesh Cricket Association Stadium',
 'M Chinnaswamy Stadium',
 'MA Chidambaram Stadium',
 'Narendra Modi Stadium',
 'Punjab Cricket Association IS Bindra Stadium',
 'Rajiv Gandhi International Stadium',
 'Sawai Mansingh Stadium',
 'Wankhede Stadium',
 'Other']

11

['Chennai Super Kings',
 'Delhi Capitals',
 'Gujarat Titans',
 'Kolkata Knight Riders',
 'Lucknow Super Giants',
 'Mumbai Indians',
 'Punjab Kings',
 'Rajasthan Royals',
 'Royal Challengers Bangalore',
 'Sunrisers Hyderabad',
 'Other']

In [135]:
onehot = ("onehot", OneHotEncoder(sparse_output=False, drop='first', categories=[
    venue_categories, team_categories, team_categories
]), ["venue", "batting_team", "bowling_team"])

scaler = ("scaler", StandardScaler(), ["count_batsmen", "count_bowlers"])

In [136]:
onehot_scaler = ColumnTransformer([onehot, scaler], remainder='passthrough')

In [178]:
train_X_transformed = onehot_scaler.fit_transform(train_X)
val_X_transformed = onehot_scaler.transform(val_X)

In [181]:
train_X
val_X

,venue,innings,batting_team,bowling_team,count_batsmen,count_bowlers
704,Eden Gardens,1,Kolkata Knight Riders,Mumbai Indians,3,4
1316,Arun Jaitley Stadium,1,Delhi Capitals,Kolkata Knight Riders,2,5
1404,Arun Jaitley Stadium,1,Kolkata Knight Riders,Delhi Capitals,3,4
701,M Chinnaswamy Stadium,2,Other,Royal Challengers Bangalore,5,4
34,Rajiv Gandhi International Stadium,1,Other,Punjab Kings,3,3
...,...,...,...,...,...,...
806,Other,1,Rajasthan Royals,Punjab Kings,3,4
1620,Other,1,Royal Challengers Bangalore,Sunrisers Hyderabad,4,3
1183,Arun Jaitley Stadium,2,Kolkata Knight Riders,Delhi Capitals,5,3
1819,Wankhede Stadium,2,Mumbai Indians,Lucknow Super Giants,2,4


,venue,innings,batting_team,bowling_team,count_batsmen,count_bowlers
1740,Other,1,Royal Challengers Bangalore,Kolkata Knight Riders,3,4
441,M Chinnaswamy Stadium,2,Punjab Kings,Royal Challengers Bangalore,6,3
1034,M Chinnaswamy Stadium,1,Royal Challengers Bangalore,Sunrisers Hyderabad,3,3
1433,Wankhede Stadium,2,Mumbai Indians,Punjab Kings,3,3
1432,Wankhede Stadium,1,Punjab Kings,Mumbai Indians,2,3
...,...,...,...,...,...,...
19,Punjab Cricket Association IS Bindra Stadium,2,Mumbai Indians,Punjab Kings,4,4
504,Other,1,Chennai Super Kings,Other,4,4
1531,Other,2,Rajasthan Royals,Kolkata Knight Riders,4,3
395,Wankhede Stadium,2,Chennai Super Kings,Mumbai Indians,4,4


In [142]:
6 - 3 + 13 + 11 + 11 - 3

35

In [180]:
train_X_transformed.shape
val_X_transformed.shape

(1515, 35)

(379, 35)

## Model selection

In [207]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

In [212]:
def evaluate_regressor(regressor, model_name):
    regressor.fit(train_X_transformed, train_y)
    predictions = regressor.predict(val_X_transformed)

    mse = mean_squared_error(val_y, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(val_y, predictions)
    r2 = r2_score(val_y, predictions)

    print(f"{model_name} Results:")
    print(f"Mean Squared Error (MSE): {mse}")
    print(f"Root Mean Squared Error (RMSE): {rmse}")
    print(f"Mean Absolute Error (MAE): {mae}")
    print(f"R-squared (R2): {r2}")
    print("\n")

In [210]:
regressors = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'ElasticNet': ElasticNet(),
    'DecisionTreeRegressor': DecisionTreeRegressor(),
    'RandomForestRegressor': RandomForestRegressor(),
    'GradientBoostingRegressor': GradientBoostingRegressor(),
    'AdaBoostRegressor': AdaBoostRegressor(),
    'SVR': SVR(),
    'MLPRegressor': MLPRegressor(),
}

for regressor in regressors.values():
    break
    evaluate_regressor(regressor)

In [242]:
from itertools import product


def grid_search_and_evaluate(regressor, param_grid, train_X, val_X, train_y, val_y, scoring_func):
    """
    Perform grid search over hyperparameters and evaluate the regressor.

    Parameters:
    - regressor: An instance of the regression model (e.g., RandomForestRegressor, MLPRegressor, etc.)
    - param_grid: A dictionary with hyperparameter names as keys and lists of possible values as values.
    - train_X, val_X, train_y, val_y: Training and validation data.
    - scoring_func: A scoring function used for evaluation (e.g., mean_squared_error, r2_score).

    Returns:
    - best_params: The best hyperparameter combination found.
    - best_score: The best score achieved.
    """

    best_params = None
    best_score = float('inf')

    # Generate all combinations of hyperparameters
    param_combinations = list(product(*param_grid.values()))

    for params in param_combinations:
        # Create a dictionary of hyperparameters
        param_dict = dict(zip(param_grid.keys(), params))

        # Set hyperparameters for the regressor
        regressor.set_params(**param_dict)

        # Train the regressor
        regressor.fit(train_X, train_y)

        # Make predictions on the validation set
        val_predictions = regressor.predict(val_X)

        # Evaluate the model using the scoring function
        score = scoring_func(val_y, val_predictions)

        # Update best score and parameters if the current combination is better
        if score < best_score:
            best_score = score
            best_params = param_dict

    return best_params, best_score

In [231]:
# Example usage with RandomForestRegressor
rf_reg = RandomForestRegressor()
rf_param_grid = {
    'n_estimators': [50, 100, 200, 400],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10, 15],
}

best_params, best_score = grid_search_and_evaluate(
    rf_reg, rf_param_grid, train_X_transformed, val_X_transformed, train_y, val_y, mean_squared_error
)

print(f'Best Hyperparameters: {best_params}')
print(f'Best Mean Squared Error: {best_score}')

Best Hyperparameters: {'n_estimators': 200, 'max_depth': 10, 'min_samples_split': 15}
Best Mean Squared Error: 128.80772042879855


In [232]:
# Example usage with DecisionTreeRegressor
dt_reg = DecisionTreeRegressor()
dt_param_grid = {
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 8],
}

best_params_dt, best_score_dt = grid_search_and_evaluate(
    dt_reg, dt_param_grid, train_X_transformed, val_X_transformed, train_y, val_y, mean_squared_error
)

print(f'Best Hyperparameters (DecisionTreeRegressor): {best_params_dt}')
print(f'Best Mean Squared Error (DecisionTreeRegressor): {best_score_dt}')

Best Hyperparameters (DecisionTreeRegressor): {'max_depth': 5, 'min_samples_split': 2, 'min_samples_leaf': 4}
Best Mean Squared Error (DecisionTreeRegressor): 122.94062115772374


In [234]:
from sklearn.linear_model import Ridge

# Example usage with Ridge
ridge_reg = Ridge()
ridge_param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0],
    'fit_intercept': [True, False],
}

best_params_ridge, best_score_ridge = grid_search_and_evaluate(
    ridge_reg, ridge_param_grid, train_X_transformed, val_X_transformed, train_y, val_y, mean_squared_error
)

print(f'Best Hyperparameters (Ridge): {best_params_ridge}')
print(f'Best Mean Squared Error (Ridge): {best_score_ridge}')

Best Hyperparameters (Ridge): {'alpha': 2.0, 'fit_intercept': True}
Best Mean Squared Error (Ridge): 122.67180145076883


In [243]:
from sklearn.linear_model import Lasso

# Example usage with Lasso
lasso_reg = Lasso()
lasso_param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0],
    'fit_intercept': [True, False],
    'selection': ['cyclic', 'random'],
}

best_params_lasso, best_score_lasso = grid_search_and_evaluate(
    lasso_reg, lasso_param_grid, train_X_transformed, val_X_transformed, train_y, val_y, mean_absolute_error
)

print(f'Best Hyperparameters (Lasso): {best_params_lasso}')
print(f'Best Mean Squared Error (Lasso): {best_score_lasso}')

Best Hyperparameters (Lasso): {'alpha': 0.1, 'fit_intercept': True, 'selection': 'random'}
Best Mean Squared Error (Lasso): 8.370123839890029


In [213]:
linear_reg = LinearRegression()
evaluate_regressor(linear_reg, 'Linear Regression')

Linear Regression Results:
Mean Squared Error (MSE): 122.76468089021485
Root Mean Squared Error (RMSE): 11.079922422572048
Mean Absolute Error (MAE): 8.459808120638503
R-squared (R2): 0.1547645910491271




In [215]:
ridge_reg = Ridge()
evaluate_regressor(ridge_reg, 'Ridge Regression')

Ridge Regression Results:
Mean Squared Error (MSE): 122.70779920050728
Root Mean Squared Error (RMSE): 11.07735524394281
Mean Absolute Error (MAE): 8.456598880285503
R-squared (R2): 0.15515622175197386




In [216]:
lasso_reg = Lasso()
evaluate_regressor(lasso_reg, 'Lasso Regression')

Lasso Regression Results:
Mean Squared Error (MSE): 122.44977761851378
Root Mean Squared Error (RMSE): 11.065702762071362
Mean Absolute Error (MAE): 8.423369953459693
R-squared (R2): 0.15693270156516592




In [217]:
elastic_net_reg = ElasticNet()
evaluate_regressor(elastic_net_reg, 'ElasticNet Regression')

ElasticNet Regression Results:
Mean Squared Error (MSE): 123.81155352040881
Root Mean Squared Error (RMSE): 11.127064011697282
Mean Absolute Error (MAE): 8.512030226463883
R-squared (R2): 0.14755686803559387




In [218]:
decision_tree_reg = DecisionTreeRegressor()
evaluate_regressor(decision_tree_reg, 'Decision Tree Regression')

Decision Tree Regression Results:
Mean Squared Error (MSE): 250.05350337144534
Root Mean Squared Error (RMSE): 15.813080135490535
Mean Absolute Error (MAE): 12.23306948109059
R-squared (R2): -0.7216195541677848




In [219]:
random_forest_reg = RandomForestRegressor()
evaluate_regressor(random_forest_reg, 'Random Forest Regression')

Random Forest Regression Results:
Mean Squared Error (MSE): 148.48588084305695
Root Mean Squared Error (RMSE): 12.185478277156664
Mean Absolute Error (MAE): 9.355765954874794
R-squared (R2): -0.022325992359708335




In [220]:
gradient_boosting_reg = GradientBoostingRegressor()
evaluate_regressor(gradient_boosting_reg, 'Gradient Boosting Regression')

Gradient Boosting Regression Results:
Mean Squared Error (MSE): 126.64990168590185
Root Mean Squared Error (RMSE): 11.253883848960848
Mean Absolute Error (MAE): 8.606364528486408
R-squared (R2): 0.12801482748281545




In [221]:
adaboost_reg = AdaBoostRegressor()
evaluate_regressor(adaboost_reg, 'AdaBoost Regression')

AdaBoost Regression Results:
Mean Squared Error (MSE): 132.89351202190326
Root Mean Squared Error (RMSE): 11.527944830797173
Mean Absolute Error (MAE): 8.864086954462971
R-squared (R2): 0.08502754076963281




In [222]:
svr_reg = SVR()
evaluate_regressor(svr_reg, 'Support Vector Regression')

Support Vector Regression Results:
Mean Squared Error (MSE): 126.2490071101913
Root Mean Squared Error (RMSE): 11.236058344018657
Mean Absolute Error (MAE): 8.541319811820363
R-squared (R2): 0.1307749885339401




In [228]:
mlp_reg = MLPRegressor()
evaluate_regressor(mlp_reg, 'MLP Regression')

MLP Regression Results:
Mean Squared Error (MSE): 146.17154249774765
Root Mean Squared Error (RMSE): 12.090142368795648
Mean Absolute Error (MAE): 9.197972925387544
R-squared (R2): -0.0063917618989333125




c:\Users\k26ra\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [224]:
def actual_vs_predicted_y(val_y, val_predictions, regressor):
    import matplotlib.pyplot as plt

    # Scatter plot
    _ = plt.scatter(val_y, val_predictions, color='blue', alpha=0.5)
    _ = plt.title(f'{regressor} Actual vs Predicted')
    _ = plt.xlabel('Actual Power Play Runs')
    _ = plt.ylabel('Predicted Power Play Runs')

    # Plotting the regression line
    _ = plt.plot([min(val_y), max(val_y)], [min(val_y), max(val_y)],
                 linestyle='--', color='red', linewidth=2)

    _ = plt.show()